In [45]:
import os

# os.environ["MODIN_ENGINE"] = "ray"
from datetime import datetime, timedelta, timezone

# import modin.pandas as pd
import pandas as pd

from utils import get_db_con

In [46]:
with get_db_con() as db_con:
    query = "SELECT * FROM items"
    # cur = db_con.execute("SELECT * FROM items")
    # results = cur.fetchall()
    df = pd.read_sql_query(query, db_con)
    # UTC is 8 hours ahead of PT
    # pt_now = datetime.now(timezone.utc) - timedelta(hours=8)
    # db_df.to_csv(f"items_{pt_now.isoformat()}.csv")
    print(len(df))

1


In [47]:
def extract_city_from_category_url(row):
    # ex category_rel_url: /category/emeryville-ca/african
    return row["category_rel_url"].split("/", 3)[2].rsplit("-", 1)[0].title()


df["city"] = df.apply(lambda row: extract_city_from_category_url(row), axis=1)
df.head()

,item_name,item_description,item_rel_url,is_item_hydrated,restaurant_name,restaurant_rating,restaurant_rel_url,category_name,category_rel_url,city
0,None,None,/store/mcdonalds-6623-san-pablo-ave/MOLzwTElTX...,0,McDonald's® (6623 San Pablo Ave),4.6,/store/mcdonalds-6623-san-pablo-ave/MOLzwTElTX...,24 Hours Food,/category/emeryville-ca/24-hours,Emeryville


In [51]:
df.groupby(["city"]).size()

city
Emeryville    1
dtype: int64

In [41]:
import sqlite3

sqlite3.threadsafety

3